In [50]:
from configparser import ConfigParser
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
import sqlalchemy
import pandas as pd

In [45]:
def get_config(filename, section='postgresql'):
    parser = ConfigParser()
    parser.read(filename)
    db_params = {}
    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            db_params[param[0]] = param[1]
    else:
        raise Exception('Section {0} not found in the {1} file'.format(section, filename))
    return db_params

In [52]:
def create_database(db):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        params = get_config(SERVER_CONFIG)

        print('Connecting to the PostgreSQL server...')
        conn = psycopg2.connect(**params)
        conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
        cur = conn.cursor()

        cur.execute('DROP DATABASE IF EXISTS {}'.format(db))
        cur.execute('CREATE DATABASE {}'.format(db))
        print('Database {} successfully created.'.format(db))
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
            print('Database connection closed.')

In [58]:
def create_tables(file_names):
    engine = None
    try:
        params = get_config(DATABASE_CONFIG)
        engine = sqlalchemy.create_engine('postgresql://' + params['user'] + ':' + params['password'] + '@' + params['host'] + ':' + params['port'] + '/' + params['database'])
        for file_name in file_names:            
            data = pd.read_csv(DATA_URL + file_name + '.csv')
            data.to_sql(file_name, engine, if_exists='replace')
            print('Table {} successfully created.'.format(file_name))
    except (Exception) as error:
        print(error)
    finally:
        if engine is not None:
            engine.dispose()
            print('Database connection closed.')

In [60]:
DB = 'pluralsight'
SERVER_CONFIG = 'server.ini'
create_database(DB)

Connecting to the PostgreSQL database...
Database pluralsight successfully created.
Database connection closed.


In [61]:
DATABASE_CONFIG = 'database.ini'
DATA_URL = 'https://raw.githubusercontent.com/aatishsuman/plural-sight-exercise/master/data/'
FILE_NAMES = ['course_tags', 'user_assessment_scores', 'user_course_views', 'user_interests']
create_tables(FILE_NAMES)

Table course_tags successfully created.
Table user_assessment_scores successfully created.
Table user_course_views successfully created.
Table user_interests successfully created.
Database connection closed.
